Functional Requirements - Load Credit Card Database (SQL)

Functional Requirement 1.1
a)  For “Credit Card System,” create a Python and PySpark SQL
    program to read/extract the following JSON files according to the
    specifications found in the mapping document.
        1. CDW_SAPP_BRANCH.JSON
        2. CDW_SAPP_CREDITCARD.JSON
        3. CDW_SAPP_CUSTOMER.JSON

Note: Data Engineers will be required to transform the data based on the
requirements found in the Mapping Document.

Hint: [You can use PySQL “select statement query” or simple Pyspark
RDD].


Req-1.2 Data loading into Database

Function Requirement 1.2
Once PySpark reads data from JSON files, and then utilizes Python,
PySpark, and Python modules to load data into RDBMS(SQL), perform the
following:
    a) Create a Database in SQL(MySQL), named “creditcard_capstone.”
    b) Create a Python and Pyspark Program to load/write the “Credit
    
Card System Data” into RDBMS(creditcard_capstone).
Tables should be created by the following names in RDBMS:
CDW_SAPP_BRANCH
CDW_SAPP_CREDIT_CARD
CDW_SAPP_CUSTOMER

!!!!!! ASK BEN ABOUT THIS STUFF !!!!!!!!


What is the difference between Spark session and context?
The SparkContext provides methods for creating RDDs, accumulators, and broadcasting variables, as well as methods for starting tasks on the executors. The SparkSession does not provide these methods, but it does provide methods for creating DataFrames and Datasets, as well as methods for reading and writing data.

ASK BEN ABOUT THIS STUFF
User interface for active sessions

from pyspark import SparkContext 
#this is need to connect to the local host to see the the active session is running

In [2]:
# Setup for getting ready to Load the database via a Spark Session and sparkSQL
# Previously CREATED the "creditcard_capstone" DATABASE #
# directly within the mySql workbench


# Import Setup Environment
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType, TimestampType


# Loading the JSON file into a spark session
import pyspark
from pyspark.sql import SparkSession #Only need one session - Importing the class object SparkSession


#### Create an active SparkSession - to be able to utilize "Spark Dataframes" (not RDBMS/previously created) ####
spark = SparkSession.builder.appName('capstone').getOrCreate() #.builder is the method in the session 



# ASK BEN ABOUT THIS STUFF
# User interface for active sessions
# # from pyspark import SparkContext # this is need to connect to the local host to see the the active session is running



# Do this at the end of the program - It is good practice to close/stop a session
'''spark = SparkSession.stop '''


# Setup to read the json file
# Both the read and json are methods on the SparkSession object
df = spark.read.json("C:\CAPSTONE\cdw_sapp_branch.json") 


#Display data etc. to ensure a sucessful load of json file
df.show()
df.printSchema()
df.describe().show()
df.columns


'''
!!!! ASK BEN ABOUT THIS !!!!
# this could cause a problem Just added?
spark.sql("SELECT count(*) from cdw_sapp_branch.json").show  '''



#SparkSession.getActiveSession() # this will display my active sesson in "local host 4040"
# http://localhost:4040/jobs/



'''Sample Code
Import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Sparkappdemo').getOrCreate()
zipdf = spark.read.json("C:/Users/zipcode1.json")
zipdf.createTempView("ziptable1")
spark.sql("SELECT count(*) from ziptable1").show
'''



+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|      BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|       Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|  1234618993|          IL|  Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|      Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|  1234663064|          FL|   Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|  1234849701|          PA|        14th Street|     19406|2018-04-18T16:51:...|
|         Paters

'Sample Code\nImport pyspark\nfrom pyspark.sql import SparkSession\nspark = SparkSession.builder.appName(\'Sparkappdemo\').getOrCreate()\nzipdf = spark.read.json("C:/Users/zipcode1.json")\nzipdf.createTempView("ziptable1")\nspark.sql("SELECT count(*) from ziptable1").show\n'

In [8]:
# Code to check and see if the SparkSession is up and running
# Code to stop the SparkSession

SparkSession.getActiveSession() # this will display my active sesson in "local host 4040"
# http://localhost:4040/jobs/


# Do this at the end of the program - It is good practice to close/stop a session
# spark = SparkSession.stop

In [3]:
df.describe()

DataFrame[summary: string, BRANCH_CITY: string, BRANCH_CODE: string, BRANCH_NAME: string, BRANCH_PHONE: string, BRANCH_STATE: string, BRANCH_STREET: string, BRANCH_ZIP: string, LAST_UPDATED: string]

In [14]:
# block of code to load branch data into dbms
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, BooleanType, DoubleType,TimestampType
schema = StructType([
      StructField("BRANCH_CODE",IntegerType(),True),
      StructField("BRANCH_NAME",StringType(),True),
      StructField("BRANCH_STREET",StringType(),True),
      StructField("BRANCH_CITY",StringType(),True),
      StructField("BRANCH_STATE",StringType(),True),
      StructField("BRANCH_ZIP",IntegerType(),True),
      StructField("BRANCH_PHONE",StringType(),True),
      StructField("LAST_UPDATED",TimestampType(),True),
  ])

df_with_schema = spark.read.schema(schema).json("C:\CAPSTONE\cdw_sapp_branch.json")

#changing the zipcode if null to transform the data
from pyspark.sql.functions import when, col, expr, concat, lit
df2= df_with_schema.withColumn("BRANCH_ZIP", when(col("BRANCH_ZIP").isNull(), 99999).otherwise(col("BRANCH_ZIP")))


'changing changing the old column branch phone, used concat to match the spec grab from 1 to 3 then grab from 4 for the next three'
'this is using the apache spark concat'
df3 = df2.withColumn(
    "BRANCH_PHONE",
    concat(
        lit("("),
        col("BRANCH_PHONE").substr(1, 3), lit(")"),
        col("BRANCH_PHONE").substr(4, 3), lit("-"),
        col("BRANCH_PHONE").substr(7, 4)
    )
)

df3.printSchema()
df3.show(3)



df3.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_BRANCH") \
  .option("user", "root") \
  .option("password", "password") \
  .save()


root
 |-- BRANCH_CODE: integer (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_ZIP: integer (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)

+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|    BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
|          1|Example Bank|     Bridle Court|        Lakeville|          MN|     55044|(123)456-5276|2018-04-18 13:51:47|
|          2|Example Bank|Washington Street|          Huntley|          IL|     60142|(123)461-8993|2018-04-18 13:51:47|
|          3|Example Bank|    Warren

#22222222222
SECOND SET OF DATA "CREDIT CARD" TO LOAD INTO THE DBMS
#22222222222

SUNDAY AND MONDAY 10/2
insert the credit card spec. directions from the document here

In [16]:
# LOAD DATA INTO A SPARK SESSION AND TAKE A LOOK AT THE DATA TYPES TO SEE WHAT I HAVE AND IF IT
# MATCHES THE INTENDED DATATYPE

###### NEED TO FIX THIS "DAY AND MONTH ARE TOGETHER BUT YEAR IS AT THE END OF THE FILE"

df = spark.read.json("C:\CAPSTONE\cdw_sapp_credit.json")

df.show()
df.printSchema()
df.describe().show()
df.columns

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|2018|
|         35|4210653349028689|123459988| 20|    3|             2|   Entertainment|            14.24|2018|
|        160|4210653349028689|123459988|  8|    7|             3|         Grocery|             56.7|2018|
|        114|4210653349028689|123459988| 19|    4|             4|   Entertainment|            59.73|2018|
|         93|4210653349028689|123459988| 10|   10|             5|             Gas|             3.59|2018|
|        164|4210653349028689|123459988| 28|    5|             6|       Education|             6.89|2018|
|        119|4210653349028689|123459988| 19|  

['BRANCH_CODE',
 'CREDIT_CARD_NO',
 'CUST_SSN',
 'DAY',
 'MONTH',
 'TRANSACTION_ID',
 'TRANSACTION_TYPE',
 'TRANSACTION_VALUE',
 'YEAR']

In [3]:
# block of code to load credit card data into dbms
# from pyspark.sql.types import StructType


## THIS IS THE ORDER OF THE DATA IN THE JSON FILE MAPPED IT TO THE BELOW ORDER BUT WILL NEED TO FIX THE DATE
'''ConnectionAbortedErrorroot
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- YEAR: long (nullable = true)'''

schema = StructType([
      StructField("TRANSACTION_ID",IntegerType(),True),
      StructField("BRANCH_CODE",IntegerType(),True),
      StructField("CREDIT_CARD_NO",StringType(),True),
      StructField("CUST_SSN",IntegerType(),True), 
      StructField("DAY",StringType(),True),
      StructField("MONTH",StringType(),True),
      StructField("YEAR",StringType(),True),
      StructField("TRANSACTION_TYPE",StringType(),True),
      StructField("TRANSACTION_VALUE",DoubleType(),True)   
  ])



df_with_schema = spark.read.schema(schema).json("C:\CAPSTONE\cdw_sapp_credit.json")

#changing the zipcode if null to transform the data
#pad the left column below with up to the length of 2 ie from 2 to 02 but if already 10 ie 2 digits does nothing
from pyspark.sql.functions import when, col, expr, concat, lit, lpad


#########      TO DO LIST      #########

# DO THIS FOR EACH COLUMN IN THE MAPPING DOCUMENT THAT HAS A DIFFERENT COLUMN NAME IN THE TARGET TABLE/DBMS COLUMN NAME
# FROM THE SOURCE FILE COLUMN NAMES
df2= df_with_schema.withColumn("CUST_CC_NO", col("CREDIT_CARD_NO"))

df2 = df2.withColumn(
    "TIMEID",
    concat(
        col("YEAR"),
        lpad(col("MONTH"), 2, "0"),
        lpad(col("DAY"), 2, "0")
    )
)

df2 = df2.drop("DAY", "MONTH", "YEAR", "CREDIT_CARD_NO")

df2.printSchema()
df2.show(3)



df2.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
  .option("user", "root") \
  .option("password", "password") \
  .save()

root
 |-- TRANSACTION_ID: integer (nullable = true)
 |-- BRANCH_CODE: integer (nullable = true)
 |-- CUST_SSN: integer (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- CUST_CC_NO: string (nullable = true)
 |-- TIMEID: string (nullable = true)

+--------------+-----------+---------+----------------+-----------------+----------------+--------+
|TRANSACTION_ID|BRANCH_CODE| CUST_SSN|TRANSACTION_TYPE|TRANSACTION_VALUE|      CUST_CC_NO|  TIMEID|
+--------------+-----------+---------+----------------+-----------------+----------------+--------+
|             1|        114|123459988|       Education|             78.9|4210653349028689|20180214|
|             2|         35|123459988|   Entertainment|            14.24|4210653349028689|20180320|
|             3|        160|123459988|         Grocery|             56.7|4210653349028689|20180708|
+--------------+-----------+---------+----------------+-----------------+------------

In [1]:
# this is to load in the customer data table in the rdbms

# LOAD DATA INTO A SPARK SESSION AND TAKE A LOOK AT THE DATA TYPES TO SEE WHAT I HAVE AND IF IT
# MATCHES THE INTENDED DATATYPE

###### NEED TO FIX THIS "DAY AND MONTH ARE TOGETHER BUT YEAR IS AT THE END OF THE FILE"

df = spark.read.json("C:\CAPSTONE\cdw_sapp_custmer.json")

df.show()
df.printSchema()
df.describe().show()
df.columns


NameError: name 'spark' is not defined

In [4]:
# block of code to load credit card data into dbms
# from pyspark.sql.types import StructType


## THIS IS THE ORDER OF THE DATA IN THE JSON FILE MAPPED IT TO THE BELOW ORDER BUT WILL NEED TO FIX THE DATE
""" root
 |-- TRANSACTION_ID: integer (nullable = true)
 |-- BRANCH_CODE: integer (nullable = true)
 |-- CUST_SSN: integer (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- CUST_CC_NO: string (nullable = true)
 |-- TIMEID: string (nullable = true)' """

schema = StructType([
      StructField("SSN",IntegerType(),True),
      StructField("FIRST_NAME",StringType(),True),
      StructField("MIDDLE_NAME",StringType(),True),
      StructField("LAST_NAME",StringType(),True), 
      StructField("CREDIT_CARD_NO",StringType(),True),
      StructField("STREET_NAME",StringType(),True),
      StructField("APT_NO",StringType(),True),
      StructField("CUST_CITY",StringType(),True),
      StructField("CUST_STATE",StringType(),True),
      StructField("CUST_COUNTRY",StringType(),True), 
      StructField("CUST_ZIP",StringType(),True),
      StructField("CUST_EMAIL",StringType(),True),
      StructField("LAST_UPDATED",TimestampType(),True),
      StructField("CUST_PHONE",StringType(),True)
  ])



df = spark.read.schema(schema).json("C:\CAPSTONE\cdw_sapp_custmer.json")

#changing the zipcode if null to transform the data
#pad the left column below with up to the length of 2 ie from 2 to 02 but if already 10 ie 2 digits does nothing
from pyspark.sql.functions import when, col, expr, concat, lit, lpad, lower, initcap, concat_ws


#########      TO DO LIST      #########

'changing changing the old column branch phone, used concat to match the spec grab from 1 to 3 then grab from 4 for the next three'
'this is using the apache spark concat'
df = df.withColumn(
    "CUST_PHONE",
    concat(
        lit("("),
        col("CUST_PHONE").substr(1, 3), lit(")"),
        col("CUST_PHONE").substr(4, 3), lit("-"),
        col("CUST_PHONE").substr(7, 4))
)

df = df.withColumn("FIRST_NAME", initcap(df["FIRST_NAME"]))
df = df.withColumn("MIDDLE_NAME", lower(df["MIDDLE_NAME"]))
df = df.withColumn("LAST_NAME", initcap(df["LAST_NAME"]))


df = df.withColumn("FULL_STREET_ADDRESS", concat_ws(", ", df["STREET_NAME"], df["APT_NO"]))

df = df.withColumn("CUST_ZIP", col("CUST_ZIP").cast(IntegerType()))

df = df.drop("STREET_NAME", "APT_NO")

df.printSchema()
df.show(3)



df.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_CUSTOMER") \
  .option("user", "root") \
  .option("password", "password") \
  .save()

root
 |-- SSN: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_ZIP: integer (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = false)

+---------+----------+-----------+---------+----------------+------------+----------+-------------+--------+-------------------+-------------------+----------+--------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  CREDIT_CARD_NO|   CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|         CUST_EMAIL|       LAST_UPDATED|CUST_PHONE| FULL_STREET_ADDRESS|
+---------+----------+-----------+---------+----------------+------------

In [38]:
# df.select("CUST_PHONE").show()
df.select("CUST_ZIP").show()

+--------+
|CUST_ZIP|
+--------+
|   39120|
|    6109|
|   60142|
|   53151|
|   79930|
|   44070|
|   22180|
|   91010|
|   48867|
|   60099|
|   44512|
|   29483|
|   79930|
|   48430|
|   49418|
|   95993|
|   33904|
|   53045|
|   23223|
|   19380|
+--------+
only showing top 20 rows



2. Functional Requirements - Application Front-End
Once data is loaded into the database, we need a front-end (console) to see/display data. For that, create a console-based Python program to satisfy System Requirements 2 (2.1 and 2.2).

Req-2.1 - Transaction Details Module
Functional Requirements 2.1
    1) Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in      descending order.
    2) Used to display the number and total values of transactions for a given type.
    3) Used to display the total number and total values of transactions for branches in a given state.


Req-2.2	- Customer Details
Functional Requirements 2.2	
    1) Used to check the existing account details of a customer.
    2) Used to modify the existing account details of a customer.
    3) Used to generate a monthly bill for a credit card number for a given month and year.
    4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order.


# INTERFACE CODE #

In [ ]:
#import pyspark
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.master('local[1]').appName('demo.com').getOrCreate()

#Need to pick between Transaction details and the customer
select=input("Choose which details you would like to see\n\t1-Transaction\n\t2-Customer\n")

#df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
#    user="root",\
#    password="password",\
#    url="jdbc:mysql://localhost:3306/classicmodels",\
#    dbtable="classicmodels.orders").load()
#df.createTempView("ziptable1")
#Need zip code, month, and year
#spark.sql("SELECT count(*) from ziptable1").show()

if select=="1": #Transaction
    option=input("\n\n\n\nSelect the option you would like to run\n\t1-Display the transactions made by " 
                 "customers living in a given zip code for a given month and year\n\t2-Display the " 
                 "number and total values of transactions for a given type\n\t3-Display the total "
                 "number and total values of transactions for branches in a given state\n")
    if option=="1":
        zip_code=input("\n\n\n\nPlease enter the zip code\n")
        month=input("\n\n\n\nPlease enter the month as MM\n") 
        year=input("\n\n\n\nPlease enter the year as YYYY\n")
        #show all transactions made by all customers in this zip code in this month in this year
        #order by day in descending order
    elif option=="2":
        #Need the type of transaction
        transaction_type=input("\n\n\n\nPlease enter the transaction type\n")
        #show the total number and total value of these transaction types
    elif option=="3":
        ##Need the state
        state=input("\n\n\n\nPlease enter the state's two letter abbreviation (ex. GA)\n")
        #sum the total number of transactions of the given state and then the total value
    else:
        print("Not an option- Start Over")
        raise
        
elif select=="2": #Customer
    option=input("\n\n\n\nSelect the option you would like to run\n\t1-Check the existing account details of a customer " 
                 "\n\t2-Modify the existing account details of a customer " 
                 "\n\t3-Generate a monthly bill for a credit card number for a given month and year "
                 "\n\t4-Display the transactions made by a customer between two dates\n")
    if option=="1": #Need something to identify this customer to show account details
        SSn=input("\n\n\n\nPlease enter your SSN\n")
        #show the account details (probably the row of this SSN)
    elif option=="2": #To modify the account
        SSn=input("\n\n\n\nPlease enter your SSN\n")
        to_modify=input("\n\n\n\nSelect the option you would like to change"
                "\n\t1-Last Name"
                "\n\t2-Street Address"
                "\n\t3-City"
                "\n\t4-State"
                "\n\t5-Country"
                "\n\t6-Zip Code"
                "\n\t7-Email"
                "\n\t8-Phone\n")
        if to_modify=="1":#last name
            change_to=input("\n\n\n\nPlease enter your new last name\n")
            column_name="LAST_NAME"
        elif to_modify=="2":#street address
            change_to=input("\n\n\n\nPlease enter your new street address as (Street, Apartment)\n")
            column_name="FULL_STREET_ADDRESS"
        elif to_modify=="3": #city
            change_to=input("\n\n\n\nPlease enter your new city\n")
            column_name="CUST_CITY"
        elif to_modify=="4":#state
            change_to=input("\n\n\n\nPlease enter your new state\n")
            column_name="CUST_STATE"
        elif to_modify=="5":#country
            change_to=input("\n\n\n\nPlease enter your new country\n")
            column_name="CUST_COUNTRY"
        elif to_modify=="6":#zip code
            change_to=input("\n\n\n\nPlease enter your new zip code\n")
            column_name="CUST_ZIP"
        elif to_modify=="7":#email
            change_to=input("\n\n\n\nPlease enter your new email\n")
            column_name="CUST_EMAIL"
        elif to_modify=="8":#phone
            change_to=input("\n\n\n\nPlease enter your new phone number\n")
            column_name="CUST_PHONE"
        else:
            print("Not an option- Start Over")
            raise
        #Now use the SSN, column_name and the change_to variable to modify the dataframe
        
    elif option=="3": #Generate a monthly bill for a credit card number for a given month/year
        CCn=input("\n\n\n\nPlease enter the credit card number\n")
        month=input("\n\n\n\nPlease enter the month as MM\n")
        year=input("\n\n\n\nPlease enter the year as YYYY\n")
        #sum the transaction values for a given CCn and month/year
    elif option=="4": #Show transactions for a customer between two dates
        SSn=input("\n\n\n\nPlease enter your SSN\n")
        date1=input("\n\n\n\nPlease enter the first date as YYYYMMDD\n")
        date2=input("\n\n\n\nPlease enter the second date as YYYYMMDD\n")
        #Order by year, month, and day in descending order
    else:
        print("Not an option- Start Over")
        raise
else:
    print("Not an option- Start Over")
    raise


2. Functional Requirements - Application Front-End
Once data is loaded into the database, we need a front-end (console) to see/display data. For that,
create a console-based Python program to satisfy System Requirements 2 (2.1 and 2.2).

Req-2.1 Transaction Details Module
Functional Requirements 2.1 

2.1.1) Used to display the transactions made by customers living in a
given zip code for a given month and year. Order by day in
descending order.

!!! THE CUST_ZIP, MONTH, AND YEAR ARE PASSED IN VIA THE INTERFACE!!!
SELECT CUST.CUST_ZIP, CREDIT.TIMEID,
	year(CREDIT.TIMEID) AS YEAR, 
    month(CREDIT.TIMEID) AS MONTH, 
    day(CREDIT.TIMEID) AS DAY,
    CREDIT.TRANSACTION_ID AS TRANS_ID,
    CREDIT.TRANSACTION_TYPE AS TRANS_TYPE,
    CREDIT.TRANSACTION_VALUE AS TRANS_VALUE
FROM cdw_sapp_customer AS CUST
    INNER JOIN cdw_sapp_credit_card AS CREDIT
    ON CUST.SSN = CREDIT.cust_SSN
WHERE CUST.CUST_ZIP = '1810'
    AND CREDIT.timeid = '20181207'
ORDER BY DAY Desc
!!!! THE CUST_ZIP, MONTH, AND YEAR ARE PASSED IN VIA THE INTERFACE!!!


!!! Data Exploration via MySQL for the above query !!!
----PRODUCES 15 DISTINCT 4 DIGIT ZIP CODES----
USE creditcard_capstone;
Select distinct CUST_ZIP from cdw_sapp_customer
where length(CUST_ZIP) < 5
order by CUST_ZIP

----PRODUCES ROWS OF DATA THAT ARE BAD ZIP CODES IE DO NOT EXIST----
USE creditcard_capstone;
Select distinct * from cdw_sapp_customer
where length(CUST_ZIP) < 5
order by CUST_ZIP
!!! Data Exploration via MySQL for the above query !!!



2.1.2) Used to display the number and total values of transactions for a given type.
!!! THE TRANSACTION_TYPE IS PASSED IN VIA THE INTERFACE!!!
!!!FOR EXAMPLE 'Grocery' Returns "6549	Grocery	$337,051.63"
SELECT  COUNT(*),
		TRANSACTION_TYPE,
        CONCAT('$', FORMAT(sum(TRANSACTION_value), 2)) AS TotalDollarValue
FROM cdw_sapp_credit_card
WHERE TRANSACTION_TYPE = 'Grocery'
GROUP BY TRANSACTION_TYPE
!!! THE TRANSACTION_TYPE IS PASSED IN VIA THE INTERFACE!!!



2.1.3) Used to display the total number and total values of transactions for branches in a given state.
!!! THE b.BRANCH_STATE IS PASSED IN VIA THE INTERFACE!!!
SELECT b.BRANCH_STATE, b.BRANCH_NAME, COUNT(c.TRANSACTION_ID) AS total_transactions, 
	   CONCAT('$', format(sum(c.TRANSACTION_value), 2)) AS total_value
FROM cdw_sapp_credit_card c
JOIN cdw_sapp_branch b ON c.BRANCH_CODE = b.BRANCH_CODE
WHERE b.BRANCH_STATE = 'GA'
GROUP BY b.BRANCH_STATE, b.BRANCH_NAME
ORDER BY total_value DESC;
!!! THE b.BRANCH_STATE IS PASSED IN VIA THE INTERFACE!!!


Req-2.2	Customer Details
Functional Requirements 2.2	
    1) Used to check the existing account details of a customer.
2) Used to modify the existing account details of a customer.
3) Used to generate a monthly bill for a credit card number for a given month and year.
4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order.


2.2.1 - Used to check the existing account details of a customer.

SELECT lAST_NAME, CUST_ZIP, SSN, TIMEID, year(TIMEID), month(TIMEID), day(TIMEID) as dayOrder
FROM cdw_sapp_customer
INNER JOIN cdw_sapp_credit_card
ON cdw_sapp_customer.SSN = cdw_sapp_credit_card.cust_SSN
WHERE cdw_sapp_customer.CUST_ZIP = '1810'
AND cdw_sapp_credit_card.timeid = '20181207'
ORDER BY dayOrder Desc
